<a href="https://www.kaggle.com/code/sjagkoo7/bmi-body-mass-index-prediction?scriptVersionId=142406941" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Introduction

In this case study , we will be working with a dataset that contains information about individual's height,weight,gender,and body mass index(BMI). The BMI is a Measure that uses your height and weight to work out if your weight is healthy. The BMI calculation divides an adult's weight in kilograms by their height in meters squared.
The dataset contains the followings columns:

* `Gender : The gender of the individual`
* `Height : The height of the individual in cm`
* `Weight : The weight of individual in kg`
* `Index : The BMI index of the individual , categorized as follows:`
    `0: Extremly Weak
    1:Weak
    2 :Normal
    3:Overweight
    4:Obesity
    5:Extremly Obesity`

# Importing Libraries

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bmi-body-mass-index/bmi_train.csv
/kaggle/input/bmi-body-mass-index/bmi_validation.csv


# Reading the Dataset

In [2]:
train=pd.read_csv('/kaggle/input/bmi-body-mass-index/bmi_train.csv')
validation=pd.read_csv('/kaggle/input/bmi-body-mass-index/bmi_validation.csv')

In [3]:
train.head(2)

,Gender,Height,Weight,Index
0,Male,161,89,4
1,Male,179,127,4


In [4]:
validation.head(2)

,Gender,Height,Weight
0,Male,161,103
1,Male,180,75
